In [3]:
import functools
import datetime

# Декоратор для логування початку та завершення редагування
def log_editing(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        action = func.__name__
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{timestamp} - {self.user.username} is starting {action}.")
        try:
            result = func(self, *args, **kwargs)
        except Exception as e:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - Error during {action}: {str(e)}")
        else:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - {self.user.username} finished {action}.")
        return result
    return wrapper

# Декоратор для перевірки авторизації
def check_authentication(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        if self.user_authenticated:
            return func(self, *args, **kwargs)
        else:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - Access denied. Please log in.")
    return wrapper

# Декоратор для перевірки прав доступу при кожному виклику методу
def check_access_rights(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        if self.user_authenticated and self.user.access_level >= self.required_access_level:
            return func(self, *args, **kwargs)
        else:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - Insufficient access rights.")
    return wrapper

# Клас для представлення користувачів
class User:
    def __init__(self, username, password, access_level):
        self.username = username
        self.password = password
        self.access_level = access_level

# Основний клас редактора
class TextEditor:
    def __init__(self, access_level=1):
        self.text = ""
        self.user_authenticated = False
        self.user = User('Guest', '', access_level)
        self.required_access_level = 2

    @log_editing
    @check_authentication
    @check_access_rights
    def input_text(self, text):
        self.text = text
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{timestamp} - {self.user.username} entered text: {self.text}")

    @log_editing
    @check_authentication
    @check_access_rights
    def save_text(self, filename):
        try:
            with open(filename, 'w') as file:
                file.write(self.text)
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - {self.user.username} saved text to {filename}")
        except Exception as e:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - Error during save: {str(e)}")

    def authenticate_user(self):
        try:
            username = input("Enter your username: ")
            password = input("Enter your password: ")
            access_level = int(input("Enter your access level: "))
        except ValueError:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - Invalid input. Access level must be an integer.")
            return

        if self.authenticate(username, password, access_level):
            self.user = User(username, password, access_level)
            self.user_authenticated = True
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - {self.user.username} authenticated with access level {access_level}.")
        else:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"{timestamp} - Authentication failed. Please check your credentials.")

    def authenticate(self, username, password, access_level):
        # Тут ви маєте реалізувати механізм автентифікації.
        # Наприклад, можна використати базу даних, файли, аутентифікацію в пам'яті тощо.
        # У цьому прикладі просто перевіряємо, чи введені логін, пароль і рівень доступу співпадають.
        return username == "Misha" and password == "pass123" and access_level == 2

    def logout_user(self):
        self.user_authenticated = False
        self.user = User('Guest', '', 1)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{timestamp} - User logged out.")

# Приклад використання
editor = TextEditor(access_level=2)

# Авторизація користувача
editor.authenticate_user()

# Спроба введення та збереження тексту авторизованим користувачем
editor.input_text("Hello, this is a sample text.")
editor.save_text("output.txt")

# Вихід (логаут) користувача
editor.logout_user()

# Спроба введення та збереження тексту після виходу користувача
editor.input_text("Text after logout.")
editor.save_text("output_after_logout.txt")


2024-01-18 10:56:49 - Misha authenticated with access level 2.
2024-01-18 10:56:49 - Misha is starting input_text.
2024-01-18 10:56:49 - Misha entered text: Hello, this is a sample text.
2024-01-18 10:56:49 - Misha finished input_text.
2024-01-18 10:56:49 - Misha is starting save_text.
2024-01-18 10:56:49 - Misha saved text to output.txt
2024-01-18 10:56:49 - Misha finished save_text.
2024-01-18 10:56:49 - User logged out.
2024-01-18 10:56:49 - Guest is starting input_text.
2024-01-18 10:56:49 - Access denied. Please log in.
2024-01-18 10:56:49 - Guest finished input_text.
2024-01-18 10:56:49 - Guest is starting save_text.
2024-01-18 10:56:49 - Access denied. Please log in.
2024-01-18 10:56:49 - Guest finished save_text.
